# imports

> Functions for getting import statements.

In [ ]:
#| default_exp cli.imports

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Any, Optional
from cjm_fasthtml_tailwind.cli.utils import discover_utility_modules
from cjm_fasthtml_tailwind.cli.factory_extraction import extract_factories_from_module
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers
from cjm_fasthtml_tailwind.cli.core_utils_discovery import get_core_utilities
from cjm_fasthtml_tailwind.cli.cli_config import LibraryConfig, get_active_config

In [ ]:
#| export
def get_recommended_imports(
    modules: Optional[List[str]] = None,  # Specific modules to include, or None for all
    config: Optional[LibraryConfig] = None  # Optional configuration
) -> List[str]:  # List of import statements
    """Get recommended import statements for using the library."""
    if config is None:
        config = get_active_config()
        
    imports = []
    
    # Add base imports from configuration
    imports.extend(config.base_imports)
    
    # Get imports for utility modules
    for module_name, module in discover_utility_modules(config):
        if modules and module_name not in modules:
            continue
            
        # Get factories and helpers from this module
        factories = extract_factories_from_module(module, module_name)
        helpers = get_module_helpers(module_name)
        
        # Combine all names
        all_names = []
        if factories:
            all_names.extend([f.name for f in factories])
        if helpers:
            all_names.extend([h.name for h in helpers])
        
        if all_names:
            import_list = ", ".join(sorted(all_names))
            # Use the configuration to get the correct package path
            module_package = config.get_utilities_package(module_name)
            imports.append(f"from {module_package} import {import_list}")
    
    # Add core utilities
    core_utils = get_core_utilities(config)
    for util in core_utils:
        imports.append(util.import_statement)
    
    return imports

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()